In [3]:
import pandas as pd
from collections import Counter
import numpy as np
import math
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/justinchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
df = pd.read_csv('downloads/the-reddit-covid-dataset-comments.csv', nrows = 1000000)

In [41]:
df.tail()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
999995,comment,he2erhk,30zyl,travelhacks,False,1632469990,https://old.reddit.com/r/TravelHacks/comments/...,I used the binaxnow rapid to get into Croatia ...,0.3182,1
999996,comment,he2erd1,2qh8b,malaysia,False,1632469987,https://old.reddit.com/r/malaysia/comments/pue...,"Hi, why is Selangor covid can't deep into with...",0.0000,-13
999997,comment,he2er4e,3jp3p,leopardsatemyface,False,1632469982,https://old.reddit.com/r/LeopardsAteMyFace/com...,She'll blame this on COVID.,NaN,2
999998,comment,he2eqyw,2qh1i,askreddit,False,1632469978,https://old.reddit.com/r/AskReddit/comments/pu...,New management that made the work terrible. I ...,0.5859,1
999999,comment,he2eqkj,2rtiv,warriors,False,1632469968,https://old.reddit.com/r/warriors/comments/pu4...,"The only hope to be rid of covid, or make it m...",0.2382,0


In [10]:
df = df.drop(['type', 'id', 'subreddit.id', 'permalink', 'score', 'subreddit.nsfw'], axis=1)
df.head()

KeyError: "['type' 'id' 'subreddit.id' 'permalink'] not found in axis"

In [124]:
sample = pd.DataFrame(data=df)

In [61]:
drop_list = []
for a in range(1000000):
    if df.at[a, 'subreddit.name'] != 'anxiety' and df.at[a, 'subreddit.name'] != 'depression' and df.at[a, 'subreddit.name'] != 'mentalhealth' and df.at[a, 'subreddit.name'] != 'suicidewatch' and df.at[a, 'subreddit.name'] != 'covid19_support':
        drop_list.append(a)

In [66]:
sample = sample.drop(drop_list, axis = 0)

In [81]:
sample_data = sample.sample(1000)
sample_data

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
865271,comment,hek9urh,2f6rwj,covid19_support,False,1632807076,https://old.reddit.com/r/COVID19_support/comme...,"At the start of the pandemic, there was limite...",0.9702,1
507743,comment,hfwjml3,2qmij,anxiety,False,1633728237,https://old.reddit.com/r/Anxiety/comments/q453...,It sounds like you know that COVID is more dan...,0.6746,2
83366,comment,hhnwmp5,2f6rwj,covid19_support,False,1634933865,https://old.reddit.com/r/COVID19_support/comme...,"Thank you, that site you linked is tremendousl...",0.6753,3
632433,comment,hfff5wa,2f6rwj,covid19_support,False,1633397273,https://old.reddit.com/r/COVID19_support/comme...,"When you say ""10 years isn't impossible"" do yo...",-0.7093,3
153349,comment,hhdnqmh,2f6rwj,covid19_support,False,1634747515,https://old.reddit.com/r/COVID19_support/comme...,I have more questions.\n\nHas the medical comm...,-0.7744,4
...,...,...,...,...,...,...,...,...,...,...
166511,comment,hhbwrg0,2f6rwj,covid19_support,False,1634709105,https://old.reddit.com/r/COVID19_support/comme...,If covid didn’t give you blood clots the vacci...,0.0000,1
279633,comment,hgury3s,2f6rwj,covid19_support,False,1634388305,https://old.reddit.com/r/COVID19_support/comme...,I was in the ER in a south Florida hospital fo...,0.7732,1
864502,comment,hekdip2,2f6rwj,covid19_support,False,1632809711,https://old.reddit.com/r/COVID19_support/comme...,"Hey buddy, check this out - there's a lot of i...",0.9325,2
624064,comment,hfgjzmm,2qmij,anxiety,False,1633425292,https://old.reddit.com/r/Anxiety/comments/q1rw...,I’ll copy and paste a comment I just left on a...,-0.9781,2


In [82]:
count = Counter()
for index, row in sample_data.iterrows():
    if row['subreddit.name'] == 'anxiety':
        count['anxiety'] += 1
    if row['subreddit.name'] == 'depression':
        count['depression'] += 1
    if row['subreddit.name'] == 'suicidewatch':
        count['SuicideWatch'] += 1
    if row['subreddit.name'] == 'mentalhealth':
        count['mentalhealth'] += 1
    if row['subreddit.name'] == 'covid19_support':
        count['COVID19_support'] += 1
count

Counter({'COVID19_support': 517,
         'anxiety': 253,
         'SuicideWatch': 75,
         'mentalhealth': 70,
         'depression': 85})

## Preprocessing 

In [84]:
import re #regex library used for substituting words

In [85]:
#removing stopwords and urls, stemming, making lowercase 
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def cleaning(text, stem=False):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [86]:
def post_cleaning(text):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ',text)

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 
    
    #remove digits
    processed_feature = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", processed_feature)

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    return processed_feature

In [87]:
#above functions combined
def preprocessing(text):
    line = cleaning(text)
    line = post_cleaning (line)
    return line

In [90]:
sample_data.body = sample_data.body.apply(lambda x: preprocessing(x))

In [91]:
sample_data.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
865271,comment,hek9urh,2f6rwj,covid19_support,False,1632807076,https://old.reddit.com/r/COVID19_support/comme...,start pandemic limited knowledge virus resulte...,0.9702,1
507743,comment,hfwjml3,2qmij,anxiety,False,1633728237,https://old.reddit.com/r/Anxiety/comments/q453...,sounds like know covid dangerous vaccine terms...,0.6746,2
83366,comment,hhnwmp5,2f6rwj,covid19_support,False,1634933865,https://old.reddit.com/r/COVID19_support/comme...,thank site linked tremendously helpful messing...,0.6753,3
632433,comment,hfff5wa,2f6rwj,covid19_support,False,1633397273,https://old.reddit.com/r/COVID19_support/comme...,say years impossible mean situations like stra...,-0.7093,3
153349,comment,hhdnqmh,2f6rwj,covid19_support,False,1634747515,https://old.reddit.com/r/COVID19_support/comme...,questions medical community said whether vacci...,-0.7744,4


In [103]:
for sentiment in sample_data['sentiment']:
    if sentiment > 0:
        sample_data['sentiment'].replace({sentiment : 'positive'}, inplace=True)
    if sentiment < 0:
        sample_data['sentiment'].replace({sentiment : 'negative'}, inplace=True)
    else:
        sample_data['sentiment'].replace({sentiment : 'neutral'}, inplace=True)

In [113]:
sample_data.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
865271,comment,hek9urh,2f6rwj,covid19_support,False,1632807076,https://old.reddit.com/r/COVID19_support/comme...,start pandemic limited knowledge virus resulte...,positive,1
507743,comment,hfwjml3,2qmij,anxiety,False,1633728237,https://old.reddit.com/r/Anxiety/comments/q453...,sounds like know covid dangerous vaccine terms...,positive,2
83366,comment,hhnwmp5,2f6rwj,covid19_support,False,1634933865,https://old.reddit.com/r/COVID19_support/comme...,thank site linked tremendously helpful messing...,positive,3
632433,comment,hfff5wa,2f6rwj,covid19_support,False,1633397273,https://old.reddit.com/r/COVID19_support/comme...,say years impossible mean situations like stra...,negative,3
153349,comment,hhdnqmh,2f6rwj,covid19_support,False,1634747515,https://old.reddit.com/r/COVID19_support/comme...,questions medical community said whether vacci...,negative,4


In [92]:
from sklearn.model_selection import train_test_split

In [109]:
 # Splits Dataset into Training and Testing set
train_size = 0.8
train_data, test_data = train_test_split(sample_data, test_size=1-train_size, random_state=42, shuffle=True)
print("Train Data size:", len(train_data))
print("Test Data size", len(test_data))

Train Data size: 800
Test Data size 200


## Vecotrizing

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [106]:
vectorizer = TfidfVectorizer(max_features=2500,
                             min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
vectorizer.fit(train_data.body.to_list())
x_train = vectorizer.transform(train_data.body.to_list())
x_test = vectorizer.transform(test_data.body.to_list())

In [115]:
print("x_train Shape:",x_train.shape)
print("x_test Shape:",x_test.shape)

x_train Shape: (800, 1323)
x_test Shape: (200, 1323)


In [101]:
from sklearn.preprocessing import LabelEncoder

In [141]:
encoder = LabelEncoder() #https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
encoder.fit(train_data.sentiment.to_list())

y_train = encoder.transform(train_data.sentiment.to_list())
y_test = encoder.transform(test_data.sentiment.to_list())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

y_train shape: (800, 1)
y_test shape: (200, 1)


In [142]:
y_test

array([[0],
       [2],
       [0],
       [0],
       [2],
       [2],
       [0],
       [0],
       [2],
       [2],
       [0],
       [0],
       [2],
       [2],
       [2],
       [1],
       [2],
       [0],
       [0],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [0],
       [0],
       [2],
       [2],
       [0],
       [2],
       [2],
       [1],
       [0],
       [0],
       [0],
       [2],
       [2],
       [2],
       [0],
       [0],
       [2],
       [0],
       [2],
       [0],
       [2],
       [0],
       [2],
       [2],
       [0],
       [0],
       [2],
       [2],
       [2],
       [0],
       [2],
       [0],
       [2],
       [2],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [2],
       [2],
       [0],
       [0],
       [0],
       [2],
       [0],
       [0],
       [2],
       [0],
       [2],
       [0],
       [2],
       [0],
    

## Linear SVM Model

In [121]:
import time
from sklearn import svm
from sklearn.metrics import classification_report

In [134]:
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(x_train, y_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(x_test)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

target_names = ['positive', 'neutral', 'negative']

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))

report = classification_report(y_test, prediction_linear, target_names=target_names, output_dict=True)

print('positive: ', report['positive'])
print('negative: ', report['negative'])
print('neutral: ', report['neutral'])

Training time: 0.193143s; Prediction time: 0.040369s
positive:  {'precision': 0.6708860759493671, 'recall': 0.5638297872340425, 'f1-score': 0.6127167630057804, 'support': 94}
negative:  {'precision': 0.628099173553719, 'recall': 0.7835051546391752, 'f1-score': 0.6972477064220184, 'support': 97}
neutral:  {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9}
